In [5]:
"""
Exploratory Data Analysis (EDA) for HAM10000_metadata
This version is Jupyter-friendly and does not require command-line arguments.
"""

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# ===================== User Configuration =====================
metadata_path = 'C:/Users/admin/Desktop/capstone project/capstone dataset/HAM10000_metadata.csv'
images_dir = 'C:/Users/admin/Desktop/capstone project/capstone dataset/HAM10000_images'

outdir = 'eda_outputs'
sample_images_count = 16

os.makedirs(outdir, exist_ok=True)

# ===================== Functions =====================

def overview(df, outdir):
    print('\n=== HEAD ===')
    print(df.head())

    print('\n=== INFO ===')
    print(df.dtypes)

    print('\n=== SHAPE ===')
    print(df.shape)

    print('\n=== MISSING VALUES ===')
    miss = df.isnull().sum().sort_values(ascending=False)
    print(miss[miss>0])
    miss.to_csv(os.path.join(outdir, 'missing_values.csv'))
    print(f"Saved missing values to {os.path.join(outdir, 'missing_values.csv')}")


def basic_stats(df, outdir):
    stats = df.describe(include='all')
    stats.to_csv(os.path.join(outdir, 'metadata_describe.csv'))
    print(f"Saved descriptive stats to {os.path.join(outdir, 'metadata_describe.csv')}")
A

def plot_class_distribution(df, outdir, col='dx'):
    counts = df[col].value_counts()
    plt.figure(figsize=(10,6))
    sns.barplot(x=counts.index, y=counts.values)
    plt.xticks(rotation=45)
    plt.title(f'Distribution of {col} (lesion diagnosis)')
    plt.ylabel('Count')
    plt.xlabel(col)
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, 'class_distribution.png'))
    plt.close()


def plot_age_histogram(df, outdir, age_col='age'):
    df_age = pd.to_numeric(df[age_col], errors='coerce')
    plt.figure(figsize=(8,5))
    sns.histplot(df_age.dropna(), bins=30)
    plt.title('Age distribution')
    plt.xlabel('Age')
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, 'age_distribution.png'))
    plt.close()


def plot_sex_counts(df, outdir, sex_col='sex'):
    counts = df[sex_col].value_counts(dropna=False)
    plt.figure(figsize=(6,4))
    sns.barplot(x=counts.index.astype(str), y=counts.values)
    plt.title('Sex counts')
    plt.ylabel('Count')
    plt.xlabel('Sex')
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, 'sex_counts.png'))
    plt.close()


def plot_localization(df, outdir, loc_col='localization'):
    top = df[loc_col].value_counts().head(20)
    plt.figure(figsize=(10,6))
    sns.barplot(y=top.index, x=top.values)
    plt.title('Top 20 lesion localizations')
    plt.xlabel('Count')
    plt.ylabel('Localization')
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, 'top_localizations.png'))
    plt.close()


def correlation_heatmap(df, outdir):
    df2 = df.copy()
    for col in df2.select_dtypes(include='object').columns:
        df2[col] = df2[col].astype('category').cat.codes
    numeric = df2.select_dtypes(include=[np.number])
    if numeric.shape[1] < 2:
        print('Not enough numeric columns for correlation heatmap')
        return
    corr = numeric.corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr, annot=False, cmap='coolwarm', center=0)
    plt.title('Correlation heatmap (object columns encoded)')
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, 'correlation_heatmap.png'))
    plt.close()


def sample_images_grid(df, images_dir, outdir, image_id_col='image_id', n=16, size=(128,128)):
    if not images_dir or not os.path.isdir(images_dir):
        print('Images directory not found; skipping image grid')
        return

    samples = []
    classes = df['dx'].unique()
    per_class = max(1, n // max(1, len(classes)))
    for c in classes:
        rows = df[df['dx'] == c]
        samples += rows[image_id_col].sample(min(per_class, len(rows))).tolist()
        if len(samples) >= n:
            break

    if len(samples) < n:
        samples += df[image_id_col].sample(n - len(samples)).tolist()

    imgs = []
    for iid in samples[:n]:
        found = None
        for ext in ['jpg','jpeg','png']:
            p = os.path.join(images_dir, f'{iid}.{ext}')
            if os.path.exists(p):
                found = p
                break
        if not found:
            for fname in os.listdir(images_dir):
                if iid in fname:
                    found = os.path.join(images_dir, fname)
                    break
        if found:
            try:
                im = Image.open(found).convert('RGB').resize(size)
                imgs.append(im)
            except:
                continue

    if not imgs:
        print('No images found; skipping image grid')
        return

    cols = int(np.sqrt(len(imgs)))
    cols = max(1, cols)
    rows = int(np.ceil(len(imgs)/cols))
    grid = Image.new('RGB', (cols*size[0], rows*size[1]), color=(255,255,255))
    for idx, im in enumerate(imgs):
        x = (idx % cols) * size[0]
        y = (idx // cols) * size[1]
        grid.paste(im, (x,y))

    grid.save(os.path.join(outdir, 'sample_image_grid.jpg'))
    print('Saved sample image grid')

# ===================== Main Execution =====================

# Load metadata
metadata = pd.read_csv(metadata_path)

# Overview & stats
overview(metadata, outdir)
basic_stats(metadata, outdir)

# Plots
if 'dx' in metadata.columns:
    plot_class_distribution(metadata, outdir)
if 'age' in metadata.columns:
    plot_age_histogram(metadata, outdir)
if 'sex' in metadata.columns:
    plot_sex_counts(metadata, outdir)
if 'localization' in metadata.columns:
    plot_localization(metadata, outdir)

correlation_heatmap(metadata, outdir)

# Sample images
sample_images_grid(metadata, images_dir, outdir, image_id_col='image_id', n=sample_images_count)

print('\nEDA complete. Check the outputs in', outdir)



=== HEAD ===
     lesion_id      image_id   dx dx_type   age   sex localization
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear

=== INFO ===
lesion_id        object
image_id         object
dx               object
dx_type          object
age             float64
sex              object
localization     object
dtype: object

=== SHAPE ===
(10015, 7)

=== MISSING VALUES ===
age    57
dtype: int64
Saved missing values to eda_outputs\missing_values.csv
Saved descriptive stats to eda_outputs\metadata_describe.csv
Saved sample image grid

EDA complete. Check the outputs in eda_outputs


In [7]:
import sys
print(sys.executable)




C:\Users\admin\anaconda3\python.exe


In [1]:
import sys
print(sys.executable)


C:\Users\admin\anaconda3\python.exe


In [3]:
!{sys.executable} -m pip install numpy==1.26.3 --force-reinstall


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.3 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.3 which is incompatible.



  Using cached numpy-1.26.3-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.3-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
      Successfully uninstalled numpy-1.26.3


In [ ]:
import os
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # for progress bar

# ============================
# 📁 Paths
# ============================
image_dir = "HAM10000_images"
metadata_path = "HAM10000_metadata.csv"
out_dir = "HAM10000_yolo_cls"

# Load metadata
metadata = pd.read_csv(metadata_path)

# ============================
# 🏷 Class Mapping
# ============================
class_map = {
    "mel": "melanoma",
    "nv": "melanocytic_nevus",
    "bcc": "basal_cell_carcinoma",
    "akiec": "actinic_keratosis",
    "bkl": "benign_keratosis",
    "df": "dermatofibroma",
    "vasc": "vascular_lesions"
}

# ============================
# 🔧 Preprocessing Functions
# ============================

# Resize with aspect ratio padding (640x640)
def resize_with_padding(image, target_size=640):
    h, w = image.shape[:2]
    scale = target_size / max(h, w)
    nh, nw = int(h * scale), int(w * scale)
    resized = cv2.resize(image, (nw, nh))
    new_img = cv2.copyMakeBorder(
        resized,
        (target_size - nh) // 2,
        (target_size - nh + 1) // 2,
        (target_size - nw) // 2,
        (target_size - nw + 1) // 2,
        cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )
    return new_img

# CLAHE (Contrast normalization)
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    merged = cv2.merge((l, a, b))
    return cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

# Noise reduction (Median Filtering)
def reduce_noise(image):
    return cv2.medianBlur(image, 3)

# ============================
# 🧩 Split Dataset (70/15/15)
# ============================
train_df, test_df = train_test_split(metadata, test_size=0.15, stratify=metadata['dx'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.176, stratify=train_df['dx'], random_state=42)
splits = {"train": train_df, "val": val_df, "test": test_df}

# ============================
# 🖼 Process and Save Images
# ============================
for split, df in splits.items():
    print(f"Processing {split} set ({len(df)} images)...")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        img_name = row["image_id"] + ".jpg"
        src = os.path.join(image_dir, img_name)
        if not os.path.exists(src):
            continue

        # --- Read and preprocess ---
        img = cv2.imread(src)
        img = resize_with_padding(img, 640)
        img = apply_clahe(img)
        img = reduce_noise(img)

        
        label = class_map[row["dx"]]
        save_dir = os.path.join(out_dir, split, label)
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, img_name)
        cv2.imwrite(save_path, img)

print(" YOLOv9-ready dataset created at:", out_dir)
print("Includes: CLAHE, Median Filtering, 640×640 normalization (no augmentation)")


Processing train set (7013 images)...


100%|██████████| 7013/7013 [13:15<00:00,  8.81it/s]   


Processing val set (1499 images)...


100%|██████████| 1499/1499 [01:44<00:00, 14.35it/s]


Processing test set (1503 images)...


 72%|███████▏  | 1084/1503 [01:18<00:28, 14.67it/s]

In [5]:
!pip install ultralytics torch torchvision torchaudio --quiet

from ultralytics import YOLO
import torch

print("✅ Torch:", torch.__version__)
print("✅ CUDA available:", torch.cuda.is_available())


✅ Torch: 2.9.0+cpu
✅ CUDA available: False


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.10.0 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.16.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
streamlit 1.37.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 12.0.0 which is incompatible.


In [1]:
from ultralytics.utils.downloads import safe_download

# Download YOLOv9 classification weights
safe_download('https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov9c-cls.pt', dir='.')


In [5]:
# ==========================
# 🧠 Train YOLOv8 on Skin Cancer Dataset
# ==========================

from ultralytics import YOLO

# Load YOLOv8 classification model (pretrained on ImageNet)
model = YOLO("yolov8n-cls.pt")  # you can try yolov8s-cls.pt for better accuracy

# Train on your preprocessed dataset
results = model.train(
    data="HAM10000_yolo_cls",  # your dataset root folder
    epochs=50,                 # you can increase to 100 for better accuracy
    imgsz=640,                 # image size (same as preprocessing)
    batch=16,                  # adjust based on your system memory
    lr0=0.001,                 # learning rate
    optimizer="Adam",          # optimizer choice (SGD/Adam)
    project="runs_skin_cancer",# save folder for all experiments
    name="yolov8_skin_cls",    # experiment name
    verbose=True
)


100%|██████████| 5.31M/5.31M [00:00<00:00, 11.7MB/s]


New https://pypi.org/project/ultralytics/8.3.225 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.5  Python-3.12.7 torch-2.9.0+cpu CPU (12th Gen Intel Core(TM) i7-1255U)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=HAM10000_yolo_cls, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs_skin_cancer, name=yolov8_skin_cls, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

train: Scanning C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\train... 7013 images, 0 corrupt: 100%|██████████| 7013/7013 [00:08<00:00, 782.48it/s]


train: New cache created: C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\train.cache


C:\Users\admin\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\val... 1499 images, 0 corrupt: 100%|██████████| 1499/1499 [00:02<00:00, 532.31it/s]


...

      50/50         0G     0.2294          5        640: 100%|██████████| 439/439 [29:21<00:00,  4.01s/it]
               classes   top1_acc   top5_acc: 100%|██████████| 47/47 [01:51<00:00,  2.37s/it]

                   all      0.846      0.999



50 epochs completed in 24.967 hours.
Optimizer stripped from runs_skin_cancer\yolov8_skin_cls\weights\last.pt, 3.0MB
Optimizer stripped from runs_skin_cancer\yolov8_skin_cls\weights\best.pt, 3.0MB

Validating runs_skin_cancer\yolov8_skin_cls\weights\best.pt...
Ultralytics 8.3.5  Python-3.12.7 torch-2.9.0+cpu CPU (12th Gen Intel Core(TM) i7-1255U)
YOLOv8n-cls summary (fused): 73 layers, 1,443,847 parameters, 0 gradients, 3.3 GFLOPs
train: C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\train... found 7013 images in 7 classes  
val: C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\val... found 1499 images in 7 classes  
test: C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\test... found 1503 images in 7 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 47/47 [01:40<00:00,  2.15s/it]


                   all      0.855      0.999
Speed: 0.0ms preprocess, 45.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs_skin_cancer\yolov8_skin_cls
Results saved to runs_skin_cancer\yolov8_skin_cls


In [9]:
from ultralytics import YOLO

# Load trained model
model = YOLO(r"C:\Users\admin\Desktop\capstone project\capstone dataset\runs_skin_cancer\yolov8_skin_cls\weights\best.pt")

# Evaluate performance on validation or test data
metrics = model.val(data="HAM10000_yolo_cls", imgsz=640, batch=16)
print(metrics)


Ultralytics 8.3.5  Python-3.12.7 torch-2.9.0+cpu CPU (12th Gen Intel Core(TM) i7-1255U)
YOLOv8n-cls summary (fused): 73 layers, 1,443,847 parameters, 0 gradients, 3.3 GFLOPs
train: C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\train... found 7013 images in 7 classes  
val: C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\val... found 1499 images in 7 classes  
test: C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\test... found 1503 images in 7 classes  


val: Scanning C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\val... 1499 images, 0 corrupt: 100%|██████████| 1499/1499 [00:00<?, ?it/s]
C:\Users\admin\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
               classes   top1_acc   top5_acc: 100%|██████████| 94/94 [01:46<00:00,  1.14s/it]


                   all      0.855      0.999
Speed: 0.0ms preprocess, 49.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\val
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000016EE9C99DF0>
curves: []
curves_results: []
fitness: 0.9266177415847778
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8545697331428528, 'metrics/accuracy_top5': 0.9986657500267029, 'fitness': 0.9266177415847778}
save_dir: WindowsPath('runs/classify/val')
speed: {'preprocess': 0.012427031635999522, 'inference': 49.87871813885445, 'loss': 0.0, 'postprocess': 0.0}
task: 'classify'
top1: 0.8545697331428528
top5: 0.9986657500267029


In [19]:
from ultralytics import YOLO

model = YOLO(r"C:\Users\admin\Desktop\capstone project\capstone dataset\runs_skin_cancer\yolov8_skin_cls\weights\best.pt")

# Predict on one test image
result = model.predict(
    source=r"C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\test\melanoma\ISIC_0030107.jpg",
    save=True,
    show=True
)

# Display predicted class and confidence
probs = result[0].probs
print(f"Predicted class: {model.names[probs.top1]} ({probs.top1conf:.2f} confidence)")



image 1/1 C:\Users\admin\Desktop\capstone project\capstone dataset\HAM10000_yolo_cls\test\melanoma\ISIC_0030107.jpg: 640x640 melanocytic_nevus 0.96, melanoma 0.03, actinic_keratosis 0.01, benign_keratosis 0.00, dermatofibroma 0.00, 112.6ms
Speed: 19.6ms preprocess, 112.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\classify\predict3
Predicted class: melanocytic_nevus (0.96 confidence)
